# Analisis exploratorio de datos, Properati_ar

***Tabla de contenidos***
1. [Vista previa y filtrados esenciales](#s1)
2. [Tipos de datos por columna y manejo de datos faltantes](#s2)

## 1. Vista previa y filtrados esenciales <a id="s1"></a>

In [3]:
import pandas as pd

In [5]:
df_crudo = pd.read_csv("ar_properties.csv")

In [16]:
df_crudo.columns

Index(['id', 'ad_type', 'start_date', 'end_date', 'created_on', 'lat', 'lon',
       'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms',
       'surface_total', 'surface_covered', 'price', 'currency', 'price_period',
       'title', 'description', 'property_type', 'operation_type'],
      dtype='object')

Primero que nada será necesario filtrar por moneda... La alta inestabilidad del peso argentino hace que el analisis se complique demasiado. Es por eso que utilizaremos las propiedades publicadas en dolares.

In [17]:
df_crudo["currency"].value_counts()

USD    726918
ARS    220873
UYU      2112
PEN        27
Name: currency, dtype: int64

In [22]:
df_dolar = df_crudo[df_crudo["currency"]=="USD"]

In [39]:
df_dolar_reordered = df_dolar[['id','operation_type',
                               'l1', 'l2', 'l3', 'l4', 'l5', 'l6'
                               ,'lat', 'lon', 
                               'price', 'property_type',
                               'rooms','bedrooms', 'bathrooms',
                               'start_date', 'end_date', 'created_on',
                               'surface_total', 'surface_covered', 'price_period',
                               'title', 'description', 'ad_type']]

In [40]:
df_dolar_reordered["l1"].value_counts()

Argentina         710629
Uruguay            15611
Estados Unidos       638
Brasil                40
Name: l1, dtype: int64

Es necesario deshacerse de las propiedades ubicadas en paises que no sean la Argentina.

In [41]:
df_dolar_reordered_ar = df_dolar_reordered[df_dolar_reordered["l1"]=="Argentina"]

In [57]:
df_dolar_reordered_ar["operation_type"].value_counts()

Venta                689144
Alquiler              14770
Alquiler temporal      6715
Name: operation_type, dtype: int64

Además, dada la baja cantidad de alquileres en dólares <font color="red">se tendrán solo en cuenta las propiedades en venta</font>.

In [58]:
df_dolar_reordered_ar_venta = df_dolar_reordered_ar[df_dolar_reordered_ar['operation_type']=="Venta"]

In [63]:
df_dolar_reordered_ar_venta.shape

(689144, 24)

## 2. Tipos de datos por columna y manejo de datos faltantes <a id="s2"></a>

Vamos a renombrar el dataframe en cada seccion nueva para simplificar la lectura...

In [72]:
df_2 = df_dolar_reordered_ar_venta.drop(columns=["l1"])

In [74]:
df_2.dtypes

id                  object
operation_type      object
l2                  object
l3                  object
l4                  object
l5                  object
l6                 float64
lat                float64
lon                float64
price              float64
property_type       object
rooms              float64
bedrooms           float64
bathrooms          float64
start_date          object
end_date            object
created_on          object
surface_total      float64
surface_covered    float64
price_period        object
title               object
description         object
ad_type             object
dtype: object

In [76]:
df_2.isnull().sum()

id                      0
operation_type          0
l2                      0
l3                  37614
l4                 533846
l5                 686258
l6                 689144
lat                101672
lon                102737
price                   0
property_type           0
rooms              356733
bedrooms           433277
bathrooms          153710
start_date              0
end_date                0
created_on              0
surface_total      395424
surface_covered    415427
price_period       456574
title                  13
description             9
ad_type                 0
dtype: int64

In [83]:
df_2["l3"].value_counts()

Mar del Plata       130129
Rosario              39979
La Plata             26869
Tigre                26445
Palermo              21656
                     ...  
Hasenkamp                1
San Eduardo              1
Villa Aberastain         1
Tolhuin                  1
Berrotarán               1
Name: l3, Length: 1020, dtype: int64

In [89]:
df_2["l3"].value_counts().head(20)

Mar del Plata         130129
Rosario                39979
La Plata               26869
Tigre                  26445
Palermo                21656
Lomas de Zamora        16196
Morón                  14225
Pilar                  13255
Ituzaingó              12645
Belgrano               12064
Almagro                12030
Escobar                11229
Villa Crespo           11115
Caballito              10475
Moreno                 10377
Córdoba                10045
Quilmes                 9855
La Matanza              9827
San Isidro              9014
General San Martín      8868
Name: l3, dtype: int64

Datos geoespaciales, nos desharemos de las columnas l4, l5 y l6 (tienen demasiados datos faltantes) y para poder tener medidas de alta presición vamos a quedarnos con latitud y longitud. 

In [104]:
df_2_stripped = df_2.drop(columns=["l4","l5","l6","price_period", "bedrooms", "operation_type"])

In [105]:
df_2_stripped.isnull().sum()

id                      0
operation_type          0
l2                      0
l3                  37614
lat                101672
lon                102737
price                   0
property_type           0
rooms              356733
bathrooms          153710
start_date              0
end_date                0
created_on              0
surface_total      395424
surface_covered    415427
title                  13
description             9
ad_type                 0
dtype: int64

In [156]:
df_2_stripped_2 = df_2_stripped.dropna(axis=0, subset=["l3","rooms","lat","lon","title","description"])

Quedaría tratar con outliers y datos numéricos faltantes.

In [157]:
df_2_stripped_2.isnull().sum()

id                      0
operation_type          0
l2                      0
l3                      0
lat                     0
lon                     0
price                   0
property_type           0
rooms                   0
bathrooms           23074
start_date              0
end_date                0
created_on              0
surface_total      105122
surface_covered     98983
title                   0
description             0
ad_type                 0
dtype: int64

In [158]:
df_2_stripped_2.describe()

,lat,lon,price,rooms,bathrooms,surface_total,surface_covered
count,291411.000000,291411.000000,2.914110e+05,291411.000000,268337.000000,186289.000000,192428.000000
mean,-34.677809,-58.947811,2.467013e+05,2.964720,1.599060,245.096586,120.935867
std,1.530090,1.920913,1.852720e+07,1.681618,0.934408,2338.495739,1380.890644
min,-54.814938,-180.000000,1.000000e+00,1.000000,1.000000,-2.000000,1.000000
25%,-34.704455,-58.617063,9.000000e+04,2.000000,1.000000,48.000000,44.000000
50%,-34.604261,-58.450006,1.400000e+05,3.000000,1.000000,76.000000,67.000000
75%,-34.553921,-58.396789,2.390000e+05,4.000000,2.000000,150.000000,118.000000
max,85.051129,-4.496101,1.000000e+10,40.000000,20.000000,190000.000000,235000.000000


In [159]:
df_2_stripped_3 = df_2_stripped_2[df_2_stripped_2["rooms"]<15]
df_2_stripped_3 = df_2_stripped_3[df_2_stripped_3["bathrooms"]<10]
df_2_stripped_3 = df_2_stripped_3[df_2_stripped_3["surface_total"]<1000]
df_2_stripped_3 = df_2_stripped_3[df_2_stripped_3["surface_covered"]<1000]
df_2_stripped_3 = df_2_stripped_3[df_2_stripped_3["surface_total"]>10]
df_2_stripped_3 = df_2_stripped_3[df_2_stripped_3["surface_covered"]>10]

In [160]:
df_2_stripped_3.shape

(168357, 18)

In [161]:
df_2_stripped_3.isnull().sum()

id                 0
operation_type     0
l2                 0
l3                 0
lat                0
lon                0
price              0
property_type      0
rooms              0
bathrooms          0
start_date         0
end_date           0
created_on         0
surface_total      0
surface_covered    0
title              0
description        0
ad_type            0
dtype: int64

In [162]:
df_2_stripped_3.to_csv(r'propiedades_arg_clean.csv', index = False)

In [3]:
import pandas as pd
pd.read_csv("propiedades_arg_clean.csv").columns

Index(['id', 'operation_type', 'l2', 'l3', 'lat', 'lon', 'price',
       'property_type', 'rooms', 'bathrooms', 'start_date', 'end_date',
       'created_on', 'surface_total', 'surface_covered', 'title',
       'description', 'ad_type'],
      dtype='object')